In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [2]:
#importing the train set
train_df=pd.read_csv("train_BRCpofr.csv")

In [3]:
train_df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv
0,1,Male,Urban,Bachelor,5L-10L,1,5,5790,More than 1,A,Platinum,64308
1,2,Male,Rural,High School,5L-10L,0,8,5080,More than 1,A,Platinum,515400
2,3,Male,Urban,Bachelor,5L-10L,1,8,2599,More than 1,A,Platinum,64212
3,4,Female,Rural,High School,5L-10L,0,7,0,More than 1,A,Platinum,97920
4,5,Male,Urban,High School,More than 10L,1,6,3508,More than 1,A,Gold,59736


In [4]:
#checking the shape
train_df.shape

(89392, 12)

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89392 entries, 0 to 89391
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              89392 non-null  int64 
 1   gender          89392 non-null  object
 2   area            89392 non-null  object
 3   qualification   89392 non-null  object
 4   income          89392 non-null  object
 5   marital_status  89392 non-null  int64 
 6   vintage         89392 non-null  int64 
 7   claim_amount    89392 non-null  int64 
 8   num_policies    89392 non-null  object
 9   policy          89392 non-null  object
 10  type_of_policy  89392 non-null  object
 11  cltv            89392 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 8.2+ MB


In [6]:
train_df.describe()

,id,marital_status,vintage,claim_amount,cltv
count,89392.000000,89392.000000,89392.000000,89392.000000,89392.000000
mean,44696.500000,0.575488,4.595669,4351.502416,97952.828978
std,25805.391969,0.494272,2.290446,3262.359775,90613.814793
min,1.000000,0.000000,0.000000,0.000000,24828.000000
25%,22348.750000,0.000000,3.000000,2406.000000,52836.000000
50%,44696.500000,1.000000,5.000000,4089.000000,66396.000000
75%,67044.250000,1.000000,6.000000,6094.000000,103440.000000
max,89392.000000,1.000000,8.000000,31894.000000,724068.000000


In [7]:
#checking for null values
train_df.isnull().sum()

id                0
gender            0
area              0
qualification     0
income            0
marital_status    0
vintage           0
claim_amount      0
num_policies      0
policy            0
type_of_policy    0
cltv              0
dtype: int64

So, the data doesnot contains any null values and we can go forward to the data cleaning steps

# TRAIN DATA CLEANING:-

In [8]:
train_df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv
0,1,Male,Urban,Bachelor,5L-10L,1,5,5790,More than 1,A,Platinum,64308
1,2,Male,Rural,High School,5L-10L,0,8,5080,More than 1,A,Platinum,515400
2,3,Male,Urban,Bachelor,5L-10L,1,8,2599,More than 1,A,Platinum,64212
3,4,Female,Rural,High School,5L-10L,0,7,0,More than 1,A,Platinum,97920
4,5,Male,Urban,High School,More than 10L,1,6,3508,More than 1,A,Gold,59736


# Checking the value counts of all the columns:-

In [9]:
train_df['gender'].value_counts()

Male      50497
Female    38895
Name: gender, dtype: int64

In [10]:
train_df['area'].value_counts()

Urban    62455
Rural    26937
Name: area, dtype: int64

In [11]:
train_df['qualification'].value_counts()

High School    46247
Bachelor       39399
Others          3746
Name: qualification, dtype: int64

In [12]:
train_df['num_policies'].value_counts()

More than 1    60263
1              29129
Name: num_policies, dtype: int64

In [13]:
train_df['policy'].value_counts()

A    56644
B    24635
C     8113
Name: policy, dtype: int64

In [14]:
train_df['type_of_policy'].value_counts()

Platinum    47796
Silver      21010
Gold        20586
Name: type_of_policy, dtype: int64

In [15]:
train_df['income'].value_counts()

5L-10L           52716
2L-5L            21158
More than 10L    13664
<=2L              1854
Name: income, dtype: int64

# Replacing all the columns value with certain alphabet for ML Models:-

In [16]:
train_df['gender']=train_df['gender'].map({'Male':0,'Female':1})

In [17]:
train_df['area']=train_df['area'].map({'Urban':0,'Rural':1})

In [18]:
train_df['qualification']=train_df['qualification'].map({'High School':0,'Bachelor':1,'Others':2})

In [19]:
train_df['num_policies']=train_df['num_policies'].map({'More than 1':0,'1':1})

In [20]:
train_df['policy']=train_df['policy'].map({'A':0,'B':1,'C':2})

In [21]:
train_df['type_of_policy']=train_df['type_of_policy'].map({'Platinum':0,'Silver':1,'Gold':2})

In [22]:
train_df['income']=train_df['income'].map({'5L-10L':0,'2L-5L':1,'More than 10L':2,'<=2L':3})

In [23]:
#confirming whether the data changed or not
train_df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv
0,1,0,0,1,0,1,5,5790,0,0,0,64308
1,2,0,1,0,0,0,8,5080,0,0,0,515400
2,3,0,0,1,0,1,8,2599,0,0,0,64212
3,4,1,1,0,0,0,7,0,0,0,0,97920
4,5,0,0,0,2,1,6,3508,0,0,2,59736


In [24]:
#dropping the columns which is not required
#train_df.drop('id',axis=1,inplace=True)

In [25]:
# Check if there are any duplicated rows
if train_df.duplicated().any():
    print("The DataFrame contains duplicates.")
else:
    print("The DataFrame does not contain duplicates.")

The DataFrame does not contain duplicates.


In [26]:
train_df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv
0,1,0,0,1,0,1,5,5790,0,0,0,64308
1,2,0,1,0,0,0,8,5080,0,0,0,515400
2,3,0,0,1,0,1,8,2599,0,0,0,64212
3,4,1,1,0,0,0,7,0,0,0,0,97920
4,5,0,0,0,2,1,6,3508,0,0,2,59736


# MACHINE LEARNING MODEL:-

In [28]:
#dividing into x an y 
x=train_df.loc[:,['id','gender','area','qualification','income','marital_status','vintage','claim_amount','num_policies','policy','type_of_policy']]
y=train_df.loc[:,'cltv']

In [29]:
#Performing the train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4)

In [32]:
#loading the model
catboost = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=5)

In [33]:
#fitting the model
catboost.fit(x_train, y_train)

0:	learn: 88756.3192646	total: 170ms	remaining: 16.9s
1:	learn: 87696.5456976	total: 194ms	remaining: 9.51s
2:	learn: 86801.9520224	total: 206ms	remaining: 6.66s
3:	learn: 86077.6072657	total: 215ms	remaining: 5.16s
4:	learn: 85489.9735540	total: 222ms	remaining: 4.22s
5:	learn: 84975.7486007	total: 229ms	remaining: 3.58s
6:	learn: 84557.7146881	total: 235ms	remaining: 3.12s
7:	learn: 84219.5465497	total: 241ms	remaining: 2.77s
8:	learn: 83968.9577060	total: 247ms	remaining: 2.49s
9:	learn: 83728.2286733	total: 254ms	remaining: 2.28s
10:	learn: 83525.8096851	total: 260ms	remaining: 2.1s
11:	learn: 83362.5273776	total: 266ms	remaining: 1.95s
12:	learn: 83235.0050940	total: 273ms	remaining: 1.83s
13:	learn: 83135.0488776	total: 278ms	remaining: 1.71s
14:	learn: 83022.1745382	total: 285ms	remaining: 1.61s
15:	learn: 82947.6204003	total: 291ms	remaining: 1.53s
16:	learn: 82878.2422730	total: 297ms	remaining: 1.45s
17:	learn: 82823.4344019	total: 304ms	remaining: 1.38s
18:	learn: 82782.4605

In [81]:
#predicting the model
y_pred = catboost.predict(x_test)

In [82]:
#checking the r2 score
metrics.r2_score(y_test, y_pred)*100

16.047741924125646

In [83]:
#checking for training set
catboost.score(x_train, y_train)

0.16887683504839313

In [84]:
#checking for testing set
catboost.score(x_test, y_pred)

1.0

In [38]:
#Performing hyper parameter tunning

In [39]:
param_grid = {
    'iterations': [50, 100, 200],
    'learning_rate': [0.1, 0.5, 1],
    'depth': [3, 4, 5]
}

In [40]:
catboost_hyper=CatBoostRegressor()

In [41]:
grid_search = GridSearchCV(catboost_hyper, param_grid, cv=5, return_train_score=True)


In [42]:
grid_search.fit(x_train, y_train)

0:	learn: 89340.1529237	total: 13ms	remaining: 635ms
1:	learn: 88276.9027408	total: 24.5ms	remaining: 588ms
2:	learn: 87444.9360822	total: 31.1ms	remaining: 487ms
3:	learn: 86759.1895445	total: 45.5ms	remaining: 523ms
4:	learn: 86179.3968758	total: 60.7ms	remaining: 547ms
5:	learn: 85703.9044907	total: 70ms	remaining: 514ms
6:	learn: 85291.3436892	total: 77.7ms	remaining: 477ms
7:	learn: 84955.9078015	total: 85.1ms	remaining: 447ms
8:	learn: 84707.3093101	total: 91.9ms	remaining: 419ms
9:	learn: 84484.5697967	total: 96.8ms	remaining: 387ms
10:	learn: 84294.6071665	total: 106ms	remaining: 374ms
11:	learn: 84150.5296724	total: 110ms	remaining: 348ms
12:	learn: 84015.6600346	total: 115ms	remaining: 327ms
13:	learn: 83907.2585489	total: 121ms	remaining: 312ms
14:	learn: 83816.3526963	total: 125ms	remaining: 293ms
15:	learn: 83748.9630697	total: 131ms	remaining: 278ms
16:	learn: 83683.1727361	total: 138ms	remaining: 267ms
17:	learn: 83624.9437393	total: 142ms	remaining: 253ms
18:	learn: 835

0:	learn: 89193.6989435	total: 6.27ms	remaining: 307ms
1:	learn: 88108.1087348	total: 51.7ms	remaining: 1.24s
2:	learn: 87259.4201812	total: 66.4ms	remaining: 1.04s
3:	learn: 86554.3577208	total: 88.9ms	remaining: 1.02s
4:	learn: 85980.5510412	total: 99ms	remaining: 891ms
5:	learn: 85494.1183741	total: 104ms	remaining: 763ms
6:	learn: 85071.9439141	total: 123ms	remaining: 756ms
7:	learn: 84728.3774368	total: 140ms	remaining: 736ms
8:	learn: 84474.4585773	total: 164ms	remaining: 748ms
9:	learn: 84244.8295838	total: 189ms	remaining: 756ms
10:	learn: 84044.7890021	total: 218ms	remaining: 774ms
11:	learn: 83900.7411168	total: 230ms	remaining: 729ms
12:	learn: 83760.5172388	total: 242ms	remaining: 688ms
13:	learn: 83645.8568397	total: 254ms	remaining: 653ms
14:	learn: 83541.7968807	total: 265ms	remaining: 618ms
15:	learn: 83473.5764326	total: 276ms	remaining: 586ms
16:	learn: 83403.7118368	total: 283ms	remaining: 549ms
17:	learn: 83343.8895255	total: 290ms	remaining: 515ms
18:	learn: 83299.

0:	learn: 84631.6816756	total: 30.1ms	remaining: 1.47s
1:	learn: 83210.0417990	total: 44.2ms	remaining: 1.06s
2:	learn: 82760.9285113	total: 62.7ms	remaining: 982ms
3:	learn: 82586.1733203	total: 74.6ms	remaining: 857ms
4:	learn: 82491.4091286	total: 87.5ms	remaining: 787ms
5:	learn: 82450.6467529	total: 104ms	remaining: 761ms
6:	learn: 82414.5384730	total: 122ms	remaining: 748ms
7:	learn: 82369.4717299	total: 138ms	remaining: 726ms
8:	learn: 82347.7594755	total: 150ms	remaining: 682ms
9:	learn: 82331.5299945	total: 161ms	remaining: 643ms
10:	learn: 82310.6536459	total: 177ms	remaining: 627ms
11:	learn: 82294.2473587	total: 191ms	remaining: 605ms
12:	learn: 82280.0538912	total: 205ms	remaining: 584ms
13:	learn: 82260.8683704	total: 220ms	remaining: 566ms
14:	learn: 82247.6870862	total: 236ms	remaining: 552ms
15:	learn: 82237.9111251	total: 259ms	remaining: 550ms
16:	learn: 82227.1703067	total: 289ms	remaining: 561ms
17:	learn: 82216.4304985	total: 300ms	remaining: 533ms
18:	learn: 8220

0:	learn: 84412.1463237	total: 17ms	remaining: 831ms
1:	learn: 82984.6516025	total: 25.8ms	remaining: 620ms
2:	learn: 82525.8061796	total: 32.4ms	remaining: 508ms
3:	learn: 82351.4047869	total: 38.7ms	remaining: 445ms
4:	learn: 82254.5448393	total: 43.4ms	remaining: 391ms
5:	learn: 82217.9205550	total: 47.5ms	remaining: 348ms
6:	learn: 82168.6418482	total: 51.8ms	remaining: 318ms
7:	learn: 82128.5092964	total: 56ms	remaining: 294ms
8:	learn: 82071.4917558	total: 60.4ms	remaining: 275ms
9:	learn: 82045.1453103	total: 65.2ms	remaining: 261ms
10:	learn: 82031.4906672	total: 69.6ms	remaining: 247ms
11:	learn: 82022.8629614	total: 74ms	remaining: 234ms
12:	learn: 82016.3375557	total: 78.7ms	remaining: 224ms
13:	learn: 82002.8918850	total: 83ms	remaining: 213ms
14:	learn: 81992.3452322	total: 87.4ms	remaining: 204ms
15:	learn: 81986.8128480	total: 91.6ms	remaining: 195ms
16:	learn: 81966.7647637	total: 96.3ms	remaining: 187ms
17:	learn: 81954.3316799	total: 101ms	remaining: 179ms
18:	learn: 

0:	learn: 82671.1040611	total: 10.2ms	remaining: 500ms
1:	learn: 82436.1971031	total: 17.2ms	remaining: 413ms
2:	learn: 82353.1951536	total: 24.1ms	remaining: 377ms
3:	learn: 82326.6124122	total: 29.1ms	remaining: 334ms
4:	learn: 82286.6337250	total: 33.3ms	remaining: 300ms
5:	learn: 82242.2219631	total: 37.5ms	remaining: 275ms
6:	learn: 82210.0554160	total: 41.7ms	remaining: 256ms
7:	learn: 82183.0854861	total: 46ms	remaining: 242ms
8:	learn: 82173.1080848	total: 50.1ms	remaining: 228ms
9:	learn: 82150.1263084	total: 54.3ms	remaining: 217ms
10:	learn: 82130.8175476	total: 58.8ms	remaining: 209ms
11:	learn: 82111.6786307	total: 62.8ms	remaining: 199ms
12:	learn: 82080.0787853	total: 66.8ms	remaining: 190ms
13:	learn: 82051.3705633	total: 70.9ms	remaining: 182ms
14:	learn: 82028.9290380	total: 75.4ms	remaining: 176ms
15:	learn: 82007.7293869	total: 79.9ms	remaining: 170ms
16:	learn: 81965.1085596	total: 84.1ms	remaining: 163ms
17:	learn: 81932.3389705	total: 88.6ms	remaining: 158ms
18:	

...

158:	learn: 76916.5301021	total: 1.18s	remaining: 304ms
159:	learn: 76901.7736238	total: 1.19s	remaining: 297ms
160:	learn: 76886.1430647	total: 1.2s	remaining: 290ms
161:	learn: 76862.5903728	total: 1.21s	remaining: 283ms
162:	learn: 76835.9674020	total: 1.21s	remaining: 276ms
163:	learn: 76805.2099480	total: 1.22s	remaining: 268ms
164:	learn: 76780.6589455	total: 1.23s	remaining: 261ms
165:	learn: 76753.6835636	total: 1.23s	remaining: 253ms
166:	learn: 76728.3590475	total: 1.24s	remaining: 245ms
167:	learn: 76698.7950576	total: 1.25s	remaining: 237ms
168:	learn: 76681.1284155	total: 1.25s	remaining: 230ms
169:	learn: 76645.3305364	total: 1.26s	remaining: 222ms
170:	learn: 76620.6639748	total: 1.27s	remaining: 215ms
171:	learn: 76600.5369042	total: 1.27s	remaining: 207ms
172:	learn: 76587.1968428	total: 1.28s	remaining: 200ms
173:	learn: 76569.6980794	total: 1.29s	remaining: 193ms
174:	learn: 76548.1122261	total: 1.29s	remaining: 185ms
175:	learn: 76533.1310221	total: 1.3s	remaining: 

133:	learn: 78245.2140128	total: 938ms	remaining: 462ms
134:	learn: 78205.0037546	total: 944ms	remaining: 455ms
135:	learn: 78164.7641793	total: 951ms	remaining: 447ms
136:	learn: 78126.3292223	total: 957ms	remaining: 440ms
137:	learn: 78106.7710549	total: 963ms	remaining: 433ms
138:	learn: 78084.1499687	total: 969ms	remaining: 425ms
139:	learn: 78055.0171636	total: 974ms	remaining: 418ms
140:	learn: 78037.0736343	total: 981ms	remaining: 411ms
141:	learn: 78017.3132900	total: 987ms	remaining: 403ms
142:	learn: 77990.2500568	total: 994ms	remaining: 396ms
143:	learn: 77960.4235117	total: 1s	remaining: 389ms
144:	learn: 77938.4782575	total: 1.01s	remaining: 382ms
145:	learn: 77910.4176760	total: 1.01s	remaining: 375ms
146:	learn: 77886.6340979	total: 1.02s	remaining: 369ms
147:	learn: 77867.1319835	total: 1.04s	remaining: 364ms
148:	learn: 77843.9847151	total: 1.05s	remaining: 360ms
149:	learn: 77816.0964562	total: 1.06s	remaining: 353ms
150:	learn: 77781.9358367	total: 1.06s	remaining: 3

86:	learn: 78744.0182968	total: 568ms	remaining: 738ms
87:	learn: 78706.7338710	total: 577ms	remaining: 734ms
88:	learn: 78683.7001166	total: 582ms	remaining: 726ms
89:	learn: 78663.3424439	total: 592ms	remaining: 724ms
90:	learn: 78637.8553821	total: 599ms	remaining: 718ms
91:	learn: 78606.2843547	total: 606ms	remaining: 712ms
92:	learn: 78588.4253046	total: 613ms	remaining: 705ms
93:	learn: 78560.5563251	total: 619ms	remaining: 698ms
94:	learn: 78541.5650034	total: 627ms	remaining: 693ms
95:	learn: 78499.2523684	total: 633ms	remaining: 686ms
96:	learn: 78470.6971926	total: 642ms	remaining: 681ms
97:	learn: 78443.2050606	total: 647ms	remaining: 674ms
98:	learn: 78421.1371314	total: 656ms	remaining: 669ms
99:	learn: 78375.3528439	total: 662ms	remaining: 662ms
100:	learn: 78348.2397950	total: 670ms	remaining: 657ms
101:	learn: 78319.7996244	total: 678ms	remaining: 651ms
102:	learn: 78272.8330319	total: 687ms	remaining: 647ms
103:	learn: 78251.3676075	total: 693ms	remaining: 640ms
104:	l

37:	learn: 82593.0872665	total: 211ms	remaining: 344ms
38:	learn: 82584.0533047	total: 216ms	remaining: 338ms
39:	learn: 82575.8432745	total: 222ms	remaining: 333ms
40:	learn: 82571.0821766	total: 227ms	remaining: 326ms
41:	learn: 82564.8055302	total: 233ms	remaining: 321ms
42:	learn: 82559.7205556	total: 239ms	remaining: 317ms
43:	learn: 82554.4159966	total: 244ms	remaining: 311ms
44:	learn: 82549.9729638	total: 249ms	remaining: 304ms
45:	learn: 82544.7329555	total: 254ms	remaining: 299ms
46:	learn: 82541.4702791	total: 261ms	remaining: 294ms
47:	learn: 82533.9666927	total: 268ms	remaining: 290ms
48:	learn: 82528.8705772	total: 273ms	remaining: 284ms
49:	learn: 82524.2459912	total: 278ms	remaining: 278ms
50:	learn: 82518.3871850	total: 284ms	remaining: 273ms
51:	learn: 82514.1081011	total: 292ms	remaining: 269ms
52:	learn: 82510.6420911	total: 298ms	remaining: 264ms
53:	learn: 82508.1748333	total: 304ms	remaining: 259ms
54:	learn: 82504.3662896	total: 309ms	remaining: 253ms
55:	learn:

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x00000205B2016E20>,
             param_grid={'depth': [3, 4, 5], 'iterations': [50, 100, 200],
                         'learning_rate': [0.1, 0.5, 1]},
             return_train_score=True)

In [43]:
print("Best hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best hyperparameters:  {'depth': 3, 'iterations': 100, 'learning_rate': 0.1}
Best score:  0.15982021364147253


In [78]:
catboost2 = CatBoostRegressor(depth= 3, iterations= 100, learning_rate=0.1)

In [86]:
catboost2.fit(x_train,y_train)

0:	learn: 88756.7806151	total: 9.93ms	remaining: 983ms
1:	learn: 87685.2523710	total: 16.4ms	remaining: 804ms
2:	learn: 86846.2367309	total: 21.7ms	remaining: 702ms
3:	learn: 86149.9751564	total: 26.7ms	remaining: 641ms
4:	learn: 85563.7170291	total: 31.6ms	remaining: 600ms
5:	learn: 85081.3754752	total: 36.7ms	remaining: 575ms
6:	learn: 84664.9343140	total: 41.5ms	remaining: 551ms
7:	learn: 84325.1728113	total: 46.4ms	remaining: 533ms
8:	learn: 84071.0046558	total: 51.4ms	remaining: 520ms
9:	learn: 83845.9648352	total: 56.4ms	remaining: 508ms
10:	learn: 83654.3490195	total: 61.8ms	remaining: 500ms
11:	learn: 83504.2383998	total: 67.1ms	remaining: 492ms
12:	learn: 83363.3603083	total: 71.7ms	remaining: 480ms
13:	learn: 83261.7422477	total: 76.8ms	remaining: 472ms
14:	learn: 83172.9816300	total: 81.6ms	remaining: 463ms
15:	learn: 83107.5563204	total: 86.4ms	remaining: 454ms
16:	learn: 83040.8325249	total: 91.5ms	remaining: 446ms
17:	learn: 82981.2140188	total: 96.5ms	remaining: 440ms
18

In [87]:
y_pred24 = catboost2.predict(x_test)

In [88]:
metrics.r2_score(y_test, y_pred24)*100

16.021581202448644

In [89]:
catboost.score(x_train, y_train)

0.16887683504839313

In [90]:
catboost.score(x_test, y_pred24)

0.9951154876854722

# TEST DATA

In [91]:
test_df=pd.read_csv("test_koRSKBP.csv")

In [92]:
test_df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy
0,89393,Female,Rural,High School,5L-10L,0,6,2134,More than 1,B,Silver
1,89394,Female,Urban,High School,2L-5L,0,4,4102,More than 1,A,Platinum
2,89395,Male,Rural,High School,5L-10L,1,7,2925,More than 1,B,Gold
3,89396,Female,Rural,Bachelor,More than 10L,1,2,0,More than 1,B,Silver
4,89397,Female,Urban,High School,2L-5L,0,5,14059,More than 1,B,Silver


In [93]:
#checking the shape
test_df.shape

(59595, 11)

In [94]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59595 entries, 0 to 59594
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              59595 non-null  int64 
 1   gender          59595 non-null  object
 2   area            59595 non-null  object
 3   qualification   59595 non-null  object
 4   income          59595 non-null  object
 5   marital_status  59595 non-null  int64 
 6   vintage         59595 non-null  int64 
 7   claim_amount    59595 non-null  int64 
 8   num_policies    59595 non-null  object
 9   policy          59595 non-null  object
 10  type_of_policy  59595 non-null  object
dtypes: int64(4), object(7)
memory usage: 5.0+ MB


In [95]:
test_df.describe()

,id,marital_status,vintage,claim_amount
count,59595.000000,59595.000000,59595.000000,59595.000000
mean,119190.000000,0.576928,4.620304,4368.504321
std,17203.738983,0.494051,2.282511,3288.132408
min,89393.000000,0.000000,0.000000,0.000000
25%,104291.500000,0.000000,3.000000,2398.000000
50%,119190.000000,1.000000,5.000000,4106.000000
75%,134088.500000,1.000000,6.000000,6104.000000
max,148987.000000,1.000000,8.000000,32534.000000


In [96]:
#checking for null values
test_df.isnull().sum()

id                0
gender            0
area              0
qualification     0
income            0
marital_status    0
vintage           0
claim_amount      0
num_policies      0
policy            0
type_of_policy    0
dtype: int64

So, the data doesnot contains any null values and we can go forward to the data cleaning steps

# TEST DATA CLEANING:-

In [97]:
test_df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy
0,89393,Female,Rural,High School,5L-10L,0,6,2134,More than 1,B,Silver
1,89394,Female,Urban,High School,2L-5L,0,4,4102,More than 1,A,Platinum
2,89395,Male,Rural,High School,5L-10L,1,7,2925,More than 1,B,Gold
3,89396,Female,Rural,Bachelor,More than 10L,1,2,0,More than 1,B,Silver
4,89397,Female,Urban,High School,2L-5L,0,5,14059,More than 1,B,Silver


# Checking the value counts of all the columns:-

In [98]:
test_df['gender'].value_counts()

Male      33641
Female    25954
Name: gender, dtype: int64

In [99]:
test_df['area'].value_counts()

Urban    41441
Rural    18154
Name: area, dtype: int64

In [100]:
test_df['qualification'].value_counts()

High School    30835
Bachelor       26353
Others          2407
Name: qualification, dtype: int64

In [101]:
test_df['num_policies'].value_counts()

More than 1    40308
1              19287
Name: num_policies, dtype: int64

In [102]:
test_df['policy'].value_counts()

A    37871
B    16448
C     5276
Name: policy, dtype: int64

In [103]:
test_df['type_of_policy'].value_counts()

Platinum    31908
Silver      14053
Gold        13634
Name: type_of_policy, dtype: int64

In [104]:
test_df['income'].value_counts()

5L-10L           35088
2L-5L            14060
More than 10L     9163
<=2L              1284
Name: income, dtype: int64

# Replacing all the columns value with certain alphabet for ML Models:-

In [105]:
test_df['gender']=test_df['gender'].map({'Male':0,'Female':1})

In [106]:
test_df['area']=test_df['area'].map({'Urban':0,'Rural':1})

In [107]:
test_df['qualification']=test_df['qualification'].map({'High School':0,'Bachelor':1,'Others':2})

In [108]:
test_df['num_policies']=test_df['num_policies'].map({'More than 1':0,'1':1})

In [109]:
test_df['policy']=test_df['policy'].map({'A':0,'B':1,'C':2})

In [110]:
test_df['type_of_policy']=test_df['type_of_policy'].map({'Platinum':0,'Silver':1,'Gold':2})

In [111]:
test_df['income']=test_df['income'].map({'5L-10L':0,'2L-5L':1,'More than 10L':2,'<=2L':3})

In [112]:
#confirming whether the data changed or not
test_df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy
0,89393,1,1,0,0,0,6,2134,0,1,1
1,89394,1,0,0,1,0,4,4102,0,0,0
2,89395,0,1,0,0,1,7,2925,0,1,2
3,89396,1,1,1,2,1,2,0,0,1,1
4,89397,1,0,0,1,0,5,14059,0,1,1


In [113]:
#dropping the columns which is not required
#test_df.drop('id',axis=1,inplace=True)

In [114]:
# Check if there are any duplicated rows
if test_df.duplicated().any():
    print("The DataFrame contains duplicates.")
else:
    print("The DataFrame does not contain duplicates.")

The DataFrame does not contain duplicates.


In [115]:
test_df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy
0,89393,1,1,0,0,0,6,2134,0,1,1
1,89394,1,0,0,1,0,4,4102,0,0,0
2,89395,0,1,0,0,1,7,2925,0,1,2
3,89396,1,1,1,2,1,2,0,0,1,1
4,89397,1,0,0,1,0,5,14059,0,1,1


# PREDICTING THE TEST DATA:-

In [117]:
#Predicting the value
final_pred = catboost2.predict(test_df)

In [118]:
final_pred

array([ 94757.20008389, 124602.2916997 ,  93893.92753576, ...,
       107779.05561488, 105391.35258208, 116965.38650049])

In [119]:
test_df['cltv']=final_pred

In [120]:
test_df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv
0,89393,1,1,0,0,0,6,2134,0,1,1,94757.200084
1,89394,1,0,0,1,0,4,4102,0,0,0,124602.291700
2,89395,0,1,0,0,1,7,2925,0,1,2,93893.927536
3,89396,1,1,1,2,1,2,0,0,1,1,86945.663505
4,89397,1,0,0,1,0,5,14059,0,1,1,141575.926495


In [121]:
#creating the sample submission file
sample_submission = test_df[['id','cltv']]
sample_submission.to_csv("sample_submission(Arkajyoti_Roy).csv",index=False)